In [1]:
import ipyfan
from ipyfan.utils import norm_fn, rgba2rgb
import numpy as np
import ipywidgets as widgets
import glob
from pathlib import Path
import os
import matplotlib.pyplot as plt
from ipyfan.iis_model import ritm_model as iis_model  # AI
from solarium import read_panel_image, load_series

ipyfan.ritm.isegm.model.is_hrnet_model
torch.nn.modules.batchnorm


In [2]:
site_zip_Path = Path("/home/franchesoni/mine/creations/phd/material/data/kayrros/solarium/titan.zip")
series = load_series(site_zip_Path, outdir='figs')[0]
num_dates = len(series['B02'])

In [3]:
validate_button = widgets.Button(
    description='Validate',
    disabled=False)
previous_button = widgets.Button(
    description='Previous',
    disabled=False)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=num_dates-1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'black'},
    orientation='horizontal'
)

def validate(b):
    global w
    out_path = Path('figs') / 'annotations' / ('date_' + str(progress.value) + '_annotation.png')
    out_path.parent.mkdir(exist_ok=True, parents=True)

    if np.sum(w.annI[..., :3]) > 0:  # only save if something was masked
        plt.imsave(str(out_path), w.annI)
    else:
        # Annotation is empty, delete the corresponding file if it exists
        if os.path.exists(out_path):
            os.remove(out_path)

    progress.value += 1

    global curr_im
    global feats
    feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
    w.reset()
    w.set_image(curr_im, None, feats)

def previous(b):
    global w
    progress.value -= 1

    global curr_im
    global feats
    feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
    w.reset()
    w.set_image(curr_im, None, feats)

validate_button.on_click(validate)
previous_button.on_click(previous)

In [4]:
feats, curr_im = read_panel_image(site_zip_Path, progress.value, normalize=True)
s = ipyfan.FullSegmenter(iis_model, curr_im, curr_ref=None, feats=feats)
w = s.w  # this is how we access internal variables
widgets.VBox([s.get_widget(), widgets.HBox([validate_button, previous_button, progress])])